In [1]:
import torch
from sklearn.model_selection import train_test_split
from models.AST import AST
import os
import librosa
import numpy as np
from torchinfo import summary
from torch import nn
from torch.optim import Adam
from Utils import create_dataloader, k_fold_cross_validation

# limit GPU usage

In [2]:
torch.cuda.set_per_process_memory_fraction(0.625)

# Read Data 

In [3]:
root = '../Data/genres_original'
genres = os.listdir(root)
x = []
y = []
length = []
sr = 16*1000
for genre in genres:
    genre_root = os.path.join(root, genre)
    audios = os.listdir(genre_root)
    for audio in audios:
        audio_path = os.path.join(genre_root, audio)
        signal, sr = librosa.load(audio_path, sr=sr)
        x.append(signal)
        length.append(len(signal))
        y.append(genres.index(genre))
min_length = min(length)
print("finsh reading data")

finsh reading data


# Conversion and Compression

In [4]:
top_db = 80
for i in range(len(x)):
    signal = x[i][:min_length]
    mel_spect = librosa.feature.melspectrogram(y=signal,sr=sr) # convert signals to mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max) # log compression
    x[i] = mel_spect/-top_db
print("finish conversion and compression")

finish conversion and compression


# Split Data

In [5]:
x = np.asarray(x)
x = x.transpose((0,2,1))
y = np.asarray(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    stratify=y,shuffle=True)
# k-fold cross validation
k = 5
xs_train, ys_train, xs_valid, ys_valid = k_fold_cross_validation(x_train,y_train,k)
print("finish splitting data")

finish splitting data


In [6]:
batch_size = 16
dataloaders_train = []
dataloaders_valid = []
for i in range(k):
    dataloaders_train.append(create_dataloader(xs_train[i], ys_train[i], batch_size=batch_size))
    dataloaders_valid.append(create_dataloader(xs_valid[i], ys_valid[i], batch_size=batch_size))
dataloader_test = create_dataloader(x_test, y_test, batch_size=batch_size)
print("finish creating dataloaders")

finish creating dataloaders


In [7]:
model = AST(10,input_fdim=x.shape[2],input_tdim=x.shape[1])
model.cuda()
loss_function = nn.CrossEntropyLoss()
opt = Adam(model.parameters(), lr=0.001)
summary(model,[(16,x.shape[1],x.shape[2])])

frequncey stride=10, time stride=10
number of patches=1116


E:\learning\master\1.2\COMP47700SA\Music-Genre-Classification\venv\lib\site-packages\timm\models\vision_transformer.py:91: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  x = F.scaled_dot_product_attention(


Layer (type:depth-idx)                        Output Shape              Param #
AST                                           [16, 10]                  --
├─VisionTransformerDistilled: 1-1             --                        2,398,160
│    └─ASTPatchEmbed: 2-1                     [16, 1116, 768]           --
│    │    └─Conv2d: 3-1                       [16, 768, 12, 93]         197,376
│    └─Dropout: 2-2                           [16, 1118, 768]           --
│    └─Sequential: 2-3                        --                        --
│    │    └─Block: 3-2                        [16, 1118, 768]           7,087,872
│    │    └─Block: 3-3                        [16, 1118, 768]           7,087,872
│    │    └─Block: 3-4                        [16, 1118, 768]           7,087,872
│    │    └─Block: 3-5                        [16, 1118, 768]           7,087,872
│    │    └─Block: 3-6                        [16, 1118, 768]           7,087,872
│    │    └─Block: 3-7                        [1

In [8]:
for i in range(10):
    print("-------epoch  {} -------".format(i + 1))
    for j in range(k):
        print(f'fold {j+1}:')
        loss_train = 0
        accuracy_train = 0
        train_size = 0
        for batch_idx, (data, target) in enumerate(dataloaders_train[j]):
            model.train()
            output = model(data)
            loss = loss_function(output, target)
            opt.zero_grad()
            loss.backward()
            opt.step()
            loss_train += loss.item()*len(data)
            accuracy = (output.argmax(1) == target).sum()
            accuracy_train += accuracy
            train_size += len(data)
        print("train set loss: {}".format(loss_train/train_size))
        print("train set accuracy: {}".format(accuracy_train /train_size))

        loss_valid = 0
        accuracy_valid = 0
        valid_size = 0
        for batch_idx, (data, target) in enumerate(dataloaders_valid[j]):
            model.eval()
            with torch.no_grad():
                output = model(data)
                loss = loss_function(output, target)
                loss_valid += loss.item()*len(data)
                accuracy = (output.argmax(1) == target).sum()
                accuracy_valid += accuracy
                valid_size += len(data)
        print("valid set loss: {}".format(loss_valid/valid_size))
        print("valid set accuracy: {}".format(accuracy_valid/valid_size))
print("finish training")

-------epoch  1 -------
fold 1:
train set loss: 2.692578125
train set accuracy: 0.10000000149011612
valid set loss: 2.3814453125
valid set accuracy: 0.10625000298023224
fold 2:
train set loss: 2.368701171875
train set accuracy: 0.0859375
valid set loss: 2.3462890625
valid set accuracy: 0.08125000447034836
fold 3:
train set loss: 2.36455078125
train set accuracy: 0.12031250447034836
valid set loss: 2.3779296875
valid set accuracy: 0.10000000149011612
fold 4:
train set loss: 2.3572265625
train set accuracy: 0.10468750447034836
valid set loss: 2.371875
valid set accuracy: 0.08749999850988388
fold 5:
train set loss: 2.342822265625
train set accuracy: 0.10625000298023224
valid set loss: 2.3400390625
valid set accuracy: 0.05625000223517418
-------epoch  2 -------
fold 1:
train set loss: 2.33037109375
train set accuracy: 0.11874999850988388
valid set loss: 2.32734375
valid set accuracy: 0.13125000894069672
fold 2:
train set loss: 2.27978515625
train set accuracy: 0.12812499701976776
valid set

KeyboardInterrupt: 